# Descenso al fondo de un cráter en Marte

In [4]:
# Definición de la función mapeo
def mapeo(x, y, nr, nc, cscala):
    r = max(0, min(nr - round(y / cscala), nr - 1))
    c = max(0, min(round(x / cscala), nc - 1))
    return r, c

# Función para cargar la matriz desde un archivo .npy
def cargar_matriz(file_path):
    return np.load(file_path)

import numpy as np
import random
import math

# Función que implementa la búsqueda codiciosa
def busqueda_codiciosa(matriz, posicion_inicial, limite_diferencia=2.0):
    filas, columnas = matriz.shape
    r, c = posicion_inicial
    trayectoria = [(r, c)]  # Almacena la trayectoria del explorador
    altura_actual = matriz[r, c]
    
    # Guarda la posición inicial para mostrar al final
    posicion_inicial_info = (r, c, altura_actual)
    
    while True:
        # Obtener los vecinos válidos (8-vecinos)
        vecinos = [(r+dr, c+dc) for dr in [-1, 0, 1] for dc in [-1, 0, 1] if (dr != 0 or dc != 0)]
        vecinos_validos = [(vr, vc) for vr, vc in vecinos if 0 <= vr < filas and 0 <= vc < columnas]

        # Seleccionar el vecino con menor altura cuya diferencia sea menor a 2 metros
        menor_vecino = None

        for vr, vc in vecinos_validos:
            altura_vecino = matriz[vr, vc]
            if altura_vecino < altura_actual and abs(altura_actual - altura_vecino) <= limite_diferencia:
                if menor_vecino is None or altura_vecino < matriz[menor_vecino[0], menor_vecino[1]]:
                    menor_vecino = (vr, vc)

        # Si no hay vecino válido, salir del bucle
        if menor_vecino is None:
            break

        # Moverse al vecino con menor altura
        r, c = menor_vecino
        altura_actual = matriz[r, c]
        trayectoria.append((r, c))

    # Retornar la posición final y altura final, junto con la posición inicial
    return posicion_inicial_info, (r, c, altura_actual)

# Función que implementa el recocido simulado
def recocido_simulado(matriz, posicion_inicial, limite_diferencia=2.0, temperatura_inicial=100, enfriamiento=0.99):
    filas, columnas = matriz.shape
    r, c = posicion_inicial
    trayectoria = [(r, c)]  # Almacena la trayectoria del explorador
    temperatura = temperatura_inicial
    altura_actual = matriz[r, c]
    
    # Guarda la posición inicial para mostrar al final
    posicion_inicial_info = (r, c, altura_actual)
    
    while temperatura > 1:
        # Obtener los vecinos válidos (8-vecinos)
        vecinos = [(r+dr, c+dc) for dr in [-1, 0, 1] for dc in [-1, 0, 1] if (dr != 0 or dc != 0)]
        vecinos_validos = [(vr, vc) for vr, vc in vecinos if 0 <= vr < filas and 0 <= vc < columnas]

        # Seleccionar un vecino al azar
        vr, vc = random.choice(vecinos_validos)
        altura_vecino = matriz[vr, vc]

        # Si la nueva posición es mejor, moverse
        if altura_vecino < altura_actual and abs(altura_actual - altura_vecino) <= limite_diferencia:
            r, c = vr, vc
            altura_actual = matriz[r, c]
        else:
            # Aceptar un peor movimiento según la probabilidad del recocido simulado
            diferencia_altura = abs(altura_vecino - altura_actual)
            if diferencia_altura <= limite_diferencia and random.uniform(0, 1) < math.exp(-diferencia_altura / temperatura):
                r, c = vr, vc
                altura_actual = matriz[r, c]

        trayectoria.append((r, c))

        # Reducir la temperatura
        temperatura *= enfriamiento

    # Retornar la posición final y altura final, junto con la posición inicial
    return posicion_inicial_info, (r, c, altura_actual)

# Función de pruebas con salida simplificada
def pruebas_algoritmos(file_path, cscala=10.045):
    # Cargar la matriz
    matriz = cargar_matriz(file_path)
    nr, nc = matriz.shape  # Número de filas y columnas en la matriz

    # Posiciones de prueba (en metros)
    posiciones_metros = [
        (3200, 5000),  # Posición cercana al centro
        (3700, 4800),  # Posición 1 cercana al fondo
        (6000, 6000),  # Posición 2 cercana al fondo
        (1000, 2000),  # Posición 1 lejana
        (3000, 9000),  # Posición 2 lejana
        (7000, 5000)   # Posición aleatoria
    ]

    for i, (pos_x, pos_y) in enumerate(posiciones_metros):
        # Usar la función mapeo para obtener los índices en la matriz
        posicion_inicial = mapeo(pos_x, pos_y, nr, nc, cscala)
        print(f"\n===== Prueba {i+1}: Posición Inicial (metros): ({pos_x}, {pos_y}) =====")

        print("\n===== Búsqueda Codiciosa =====")
        inicial_codiciosa, final_codiciosa = busqueda_codiciosa(matriz, posicion_inicial)
        print(f"Posición inicial: {inicial_codiciosa[:2]}, Altura inicial: {inicial_codiciosa[2]} metros")
        print(f"Posición final: {final_codiciosa[:2]}, Altura final: {final_codiciosa[2]} metros\n")

        print("===== Recocido Simulado =====")
        inicial_recocido, final_recocido = recocido_simulado(matriz, posicion_inicial)
        print(f"Posición inicial: {inicial_recocido[:2]}, Altura inicial: {inicial_recocido[2]} metros")
        print(f"Posición final: {final_recocido[:2]}, Altura final: {final_recocido[2]} metros")



# Llamar la función de prueba
file_path = 'crater_map.npy'  # Reemplaza con la ruta correcta
pruebas_algoritmos(file_path)



===== Prueba 1: Posición Inicial (metros): (3200, 5000) =====

===== Búsqueda Codiciosa =====
Posición inicial: (579, 319), Altura inicial: 19.622644042968968 metros
Posición final: (584, 301), Altura final: 1.3097705078127182 metros

===== Recocido Simulado =====
Posición inicial: (579, 319), Altura inicial: 19.622644042968968 metros
Posición final: (572, 310), Altura final: 17.236115722656468 metros

===== Prueba 2: Posición Inicial (metros): (3700, 4800) =====

===== Búsqueda Codiciosa =====
Posición inicial: (599, 368), Altura inicial: 101.19197021484396 metros
Posición final: (596, 357), Altura final: 88.61055908203147 metros

===== Recocido Simulado =====
Posición inicial: (599, 368), Altura inicial: 101.19197021484396 metros
Posición final: (600, 350), Altura final: 68.51713378906271 metros

===== Prueba 3: Posición Inicial (metros): (6000, 6000) =====

===== Búsqueda Codiciosa =====
Posición inicial: (480, 597), Altura inicial: 151.5053857421877 metros
Posición final: (483, 60

## Analisis

### Greedy Search
- Hasta donde es capaz el explorador de llegar? 
En las pruebas realizadas, el algoritmo de búsqueda codiciosa mostró un comportamiento mixto:

En algunas pruebas, como la Prueba 1, logró llevar al explorador desde una altura inicial de 19.6 metros hasta una altura final de 1.3 metros, lo que significa que logró descender significativamente.
En otras pruebas, como la Prueba 3 y Prueba 4, apenas logró moverse, descendiendo solo unos pocos metros o quedándose en la misma posición (Prueba 4).

- Prueben su algoritmo con otras cinco posiciones cercanas y lejanas al fondo del cráter. ¿Qué tan bueno es el algoritmo para llegar al fondo del cráter es los casos probados?

El algoritmo de búsqueda codiciosa funcionó relativamente bien en posiciones donde ya había un gradiente descendente claro, como en la Prueba 1 (donde bajó hasta 1.3 metros) y la Prueba 2 (donde bajó a 88.6 metros). Sin embargo, en otras pruebas donde el terreno es más plano o el gradiente es menos claro, como la Prueba 3 y Prueba 4, el algoritmo no logró moverse significativamente o no movió al explorador del todo.

### Recocido Simulado

- ¿Qué algoritmo logra llegar más profundo en el cráter? 

En la mayoría de las pruebas, el algoritmo de recocido simulado fue capaz de llevar al explorador más profundo que la búsqueda codiciosa:

En la Prueba 2, el recocido simulado llevó al explorador desde una altura inicial de 101.1 metros a una altura final de 68.5 metros, superando la búsqueda codiciosa que llegó solo a 88.6 metros.
En la Prueba 5, también superó a la búsqueda codiciosa, logrando una altura final de 138.05 metros frente a los 139.37 metros de la búsqueda codiciosa.

- ¿Recomendarían a los ingenieros del robot utilizar alguno de estos algoritmos?

 El recocido simulado parece ser una mejor opción para explorar cráteres, especialmente en terrenos donde no hay un gradiente de altura claro o el descenso no es obvio. Este algoritmo puede explorar rutas menos óptimas inicialmente, pero eventualmente encuentra mejores posiciones más profundas. Aunque no es infalible (como en las pruebas 4 y 6), en la mayoría de los casos supera a la búsqueda codiciosa.